## CAS739 Workshop 2: Genetic Algorithms


<div class="alert alert-success">
    <h3>Exercise 1</h3>

Complete the following tournament selection definition which uses a tournament size of `t_size=3`. Plot the selected individuals and compare it to truncation selection and fitness proportionate selection.
</div>

## tournament

In [ ]:
def tournament_selection(population, fitness, t_size=3):
    selected = []
    indices = rng.choice(len(population), size=t_size)
    for j in indices:
        fitness_subset = fitness[j]
    
    winner_index = indices[np.argmax(fitness_subset)]
    
    return population[winner_index], fitness[winner_index]

In [ ]:
t_fits = np.zeros(len(efit))
t_inds = []
for i in range(len(efit)):
    p, f = tournament_selection(population, fitness)
    t_fits[i] = f
    t_inds.append(p) #skelly
    
print(t_inds)

In [ ]:
plt.scatter(fitness, np.zeros(len(fitness)), color='k', label='all')
plt.scatter(efit, np.ones(len(efit)), color='b', label='truncation')
plt.scatter(fp_fits, 2*np.ones(len(fp_fits)), color='r', label='FP')
plt.scatter(t_fits, 3*np.ones(len(t_fits)), color='y', label='tournament')
plt.legend();
ax = plt.gca()
ax.yaxis.set_tick_params(labelleft=False)

print(fp_fits)
print(t_fits)

It is difficult to represent diversity for the TSP, but below there are two different visualizations of the paths present in a population. The first maps all paths on the map, and the second plots a heatmap of the links between different city indices. A diverse population will present a variety of links, so the distribution in the heatmap should be closer to uniform for a diverse selection scheme.

<div class="alert alert-success">
    <h3>Exercise 2</h3>

Using the following visualizations, compare the diversity of the selected population from tournament selection to that of truncation selection. Do you notice a change in the distribution of links present?
</div>

In [ ]:
def plot_paths(pop):
    q=np.linspace(0.0, 1.01, len(pop))
    cmap = plt.colormaps['jet']
    cmaplist = [cmap(x) for x in q]
    plt.scatter(cities[:, 0], cities[:, 1])
    for i in range(len(pop)):
        ind = pop[i]
        plt.plot(cities[ind, 0], cities[ind, 1], color=cmaplist[i], alpha=0.1)

In [ ]:
plt.scatter(fitness, np.zeros(len(fitness)), color='k', label='all')
plt.scatter(efit, np.ones(len(efit)), color='b', label='truncation')
plt.scatter(fp_fits, 2*np.ones(len(fp_fits)), color='r', label='FP')
plt.scatter(t_fits, 3*np.ones(len(t_fits)), color='g', label='tournament')
plt.legend();

In [ ]:
plot_paths(population)

In [ ]:
plot_paths(elites)

In [ ]:
plot_paths(t_inds)

YES.

In [ ]:
!pip install pymoo

In [ ]:
from pymoo.operators.crossover import erx

print("Parents")
print(population[0], population[1])
print("Adjacency Matrix 1")
print(erx.calc_adjency_matrix(population[0]))
print("Adjacency Matrix 2")
print(erx.calc_adjency_matrix(population[1]))
print("Child")
erx.erx(population[0], population[1])

<div class="alert alert-success">
    <h3>Exercise 3</h3>

Complete the following mutation operator definition which inverts a single random pair of cities.
</div>

In [ ]:
def mutate(ind):
    
    mutated = ind.copy()
    
    index_1, index_2 = random.sample(range(len(mutated)), 2)
    
    if index_1 < index_2:
        mutated[index_1:index_2+1] = reversed(mutated[index_1:index_2+1])
    else:
        mutated[index_1:index_2+1] = reversed(mutated[index_1:index_2+1])
        
    return mutated

In [ ]:
print(population[0], population[1])
child = erx.erx(population[0], population[1])
print(child)
kid = mutate(child)
print(kid)

## <a id="ga"></a>The Genetic Algorithm

Now that we have all the different parts, we can combine them in the full genetic algorithm. We'll use two different selection methods: first, a truncation selection of the best few individuals which will pass directly to the next population. This is known as **elitism** and is done to preserve the best solutions between generations. Then, we'll use tournament selection to select parents for crossover. Finally, we'll mutate the result from crossover and pass this new individual into the next population.

In [ ]:
def ga_step(population):
    fitness = evaluate(population, d)
    next_pop, _ = truncation_selection(population, fitness)
    while len(next_pop) < len(population):
        parent1, _ = tournament_selection(population, fitness)
        parent2, _ = tournament_selection(population, fitness)
        child = erx.erx(parent1, parent2)
        child = mutate(child)
        next_pop = np.concatenate((next_pop, [child]))
    return next_pop, fitness

In [ ]:
n_cities = 20
cities = np.random.rand(n_cities, 2)
d = np.zeros((n_cities, n_cities))
for i in range(n_cities):
    for j in range(i):
        d[i, j] = np.sqrt((cities[i, 0] - cities[j, 0])**2 + (cities[i,1] - cities[j, 1])**2)
        d[j, i] = d[i, j]

In [ ]:
n_population = 100
n_gen = 100
population = np.array([rng.permutation(n_cities) for i in range(n_population)])
minfit = np.zeros(n_gen)
for i in range(n_gen):
    population, fitness = ga_step(population)
    minfit[i] = np.min(fitness)
    if i > 2 and minfit[i] < minfit[i-1]:
        print(i, minfit[i])

In [ ]:
plt.plot(minfit);

<div class="alert alert-success">
    <h3>Exercise 4</h3>

Study the impact of the crossover operator. Change the definition of the genetic algorithm to only use mutation from a single parent. How does this compare to using the ERX?
</div>

In [ ]:
def ga_step_2(population):
    fitness = evaluate(population, d)
    next_pop, _ = truncation_selection(population, fitness)
    while len(next_pop) < len(population):
        parent, _ = tournament_selection(population, fitness)
        child = mutate_2(parent)
        next_pop = np.concatenate((next_pop, [child]))
    return next_pop, fitness

In [ ]:
n_cities = 20
cities = np.random.rand(n_cities, 2)
d = np.zeros((n_cities, n_cities))
for i in range(n_cities):
    for j in range(i):
        d[i, j] = np.sqrt((cities[i, 0] - cities[j, 0])**2 + (cities[i,1] - cities[j, 1])**2)
        d[j, i] = d[i, j]

In [ ]:
n_population = 100
n_gen = 200
population = np.array([rng.permutation(n_cities) for i in range(n_population)])
minfit = np.zeros(n_gen)
for i in range(n_gen):
    population, fitness = ga_step(population)
    minfit[i] = np.min(fitness)
    if i > 2 and minfit[i] < minfit[i-1]:
        print(i, minfit[i])

In [ ]:
plt.plot(minfit);

When convergence occurs, the fitness value is better.

<div class="alert alert-success">
    <h3>Exercise 5</h3>

Study two hyperparameters of the genetic algorithm: the population size and the number of elites. How do these affect the search? Are elites necessary?
</div>

## 1. population size


In [ ]:
def ga_step(population):
    fitness = evaluate(population, d)
    next_pop, _ = truncation_selection(population, fitness)
    while len(next_pop) < len(population):
        parent1, _ = tournament_selection(population, fitness)
        parent2, _ = tournament_selection(population, fitness)
        child = erx.erx(parent1, parent2)
        child = mutate(child)
        next_pop = np.concatenate((next_pop, [child]))
    return next_pop, fitness

In [ ]:
n_cities = 20
cities = np.random.rand(n_cities, 2)
d = np.zeros((n_cities, n_cities))
for i in range(n_cities):
    for j in range(i):
        d[i, j] = np.sqrt((cities[i, 0] - cities[j, 0])**2 + (cities[i,1] - cities[j, 1])**2)
        d[j, i] = d[i, j]

In [ ]:
n_population = 200
n_gen = 100
population = np.array([rng.permutation(n_cities) for i in range(n_population)])
minfit_200 = np.zeros(n_gen)
for i in range(n_gen):
    population, fitness = ga_step(population)
    minfit_200[i] = np.min(fitness)
    if i > 2 and minfit_200[i] < minfit_200[i-1]:
        print(i, minfit_200[i])

In [ ]:
n_population = 500
n_gen = 100
population = np.array([rng.permutation(n_cities) for i in range(n_population)])
minfit_500 = np.zeros(n_gen)
for i in range(n_gen):
    population, fitness = ga_step(population)
    minfit_500[i] = np.min(fitness)
    if i > 2 and minfit_500[i] < minfit_500[i-1]:
        print(i, minfit_500[i])

In [ ]:
plt.plot(minfit_200);
plt.plot(minfit_500);
plt.plot(minfit);

In this experiment, the performance of a population size of 500 is superior to that of 200, which, in turn, is better than 100. With more individuals participating in evolution, it becomes easier to find global or near-optimal solutions, but larger population sizes require more computational resources.

## 2. The number of elites

In [ ]:
def truncation_selection(population, fitness, p=0.2):
    n_elites = int(np.floor(len(population) * p))
    elites = np.argsort(fitness)[:n_elites]
    return population[elites], fitness[elites]

In [ ]:
def ga_step(population):
    fitness = evaluate(population, d)
    next_pop, _ = truncation_selection(population, fitness)
    while len(next_pop) < len(population):
        parent1, _ = tournament_selection(population, fitness)
        parent2, _ = tournament_selection(population, fitness)
        child = erx.erx(parent1, parent2)
        child = mutate(child)
        next_pop = np.concatenate((next_pop, [child]))
    return next_pop, fitness

In [ ]:
n_cities = 20
cities = np.random.rand(n_cities, 2)
d = np.zeros((n_cities, n_cities))
for i in range(n_cities):
    for j in range(i):
        d[i, j] = np.sqrt((cities[i, 0] - cities[j, 0])**2 + (cities[i,1] - cities[j, 1])**2)
        d[j, i] = d[i, j]

In [ ]:
n_population = 100
n_gen = 100
population = np.array([rng.permutation(n_cities) for i in range(n_population)])
minfit_p_0_4 = np.zeros(n_gen)
for i in range(n_gen):
    population, fitness = ga_step(population)
    minfit_p_0_4[i] = np.min(fitness)
    if i > 2 and minfit_p_0_4[i] < minfit_p_0_4[i-1]:
        print(i, minfit_p_0_4[i])

In [ ]:
plt.plot(minfit_p_0_4);
plt.plot(minfit);

In comparison, a choice of p=0.2 is a good one, as having elites that are too large or too small is not favorable.

Elites serve as a beneficial component within genetic algorithms, as they contribute to the retention of high-quality solutions, the acceleration of convergence, and the preservation of genetic diversity. Although their presence is not absolutely essential, their incorporation frequently enhances the resilience and effectiveness of genetic algorithms.

<div class="alert alert-info">
    <h3>Discussion</h3>

In the Genetic Algorithm, how many evaluations were performed? Is this the same as the total number of permutations explored? Roughly, how many permutations were calculated compared to the total number of possible city permutations? Has the GA converged on the best possible solution? When should the GA stop?
</div>

Genetic algorithms are engineered to efficiently navigate through a subset of the vast number of possible permutations, rather than exhaustively evaluating every conceivable permutation. These algorithms strive to approximate the optimal solution by iteratively exploring a diverse range of permutations in each generation and using selection, crossover, and mutation operations to progress towards an optimal or near-optimal solution. Typically, genetic algorithms explore a significantly smaller fraction of the total number of permutations, which can be astronomical in size.

Yes.

If the fitness of the top-performing individual in the population shows little improvement over a specific number of generations (indicating a plateau), or if it meets a predefined threshold, it is a valid point to conclude and terminate the Genetic Algorithm. This method ensures that the algorithm halts when it seems to have reached a state of convergence.